In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

In [7]:
# Random seed
np.random.seed(1024)
torch.manual_seed(1024)

In [8]:
# GPU setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
parallel = True
num_workers = 0

In [9]:
# data setting
columns = [f"col_{i+1}" for i in range(501)] # 65*501
window_size=501
slide_size=501


# Data main folder
dirc = "E:/external_data/Experiment4/Spectrogram_data_csv_files/CSI_data"

# Saving path
PATH = 'C://Users/Creator/Script/Python/Project/irs_toolbox' # '.'

# Training setting
bsz = 64
pre_train_epochs = 500
fine_tune_epochs = 300

exp_name = 'Encoder_64-128-512-64-7_pretrained_on_exp4csi'

In [10]:
from data.spectrogram import import_data
from data.process_data import label_encode, create_dataloaders

def prepare_dataloader():
    X,y  = import_data(dirc,columns=columns,window_size=window_size,slide_size=slide_size)
    X = X.reshape(*X.shape,1).transpose(0,3,1,2)
    y,lb = label_encode(y)
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
    train_loader, test_loader = create_dataloaders(X_train, y_train, X_test, y_test, train_batch_sizes=bsz, test_batch_sizes=200, num_workers=num_workers)
    return train_loader, test_loader,lb


train_loader, test_loader, lb = prepare_dataloader()

Import Data


In [ ]:
from models import Lambda

class Encoder(nn.Module):
    """
    Encoder for spectrogram (1,65,65), 3 layer
    """
    def __init__(self,num_filters):
        super(Encoder, self).__init__()
        l1,l2,l3,l4 = num_filters
        ### 1st ###
        self.conv1 = nn.Conv2d(1,l1,kernel_size=(5,5),stride=(2,2))
        self.norm1 = nn.BatchNorm2d(l1)
        self.actv1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d((1,2))
        ### 2nd ###
        self.conv2 = nn.Conv2d(l1,l2,kernel_size=(4,4),stride=(2,2))
        self.norm2 = nn.BatchNorm2d(l2)
        self.actv2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d((1,2))
        ### 3rd ###
        self.conv3 = nn.Conv2d(l2,l3,kernel_size=(3,3),stride=(2,2))
        self.norm3 = nn.BatchNorm2d(l3)
        self.actv3 = nn.ReLU()
        self.pool3 = nn.AdaptiveAvgPool2d((2,2))# nn.MaxPool2d((1,2)) 
        ### 4th ###
#         self.conv4 = nn.Conv2d(l3,l4,kernel_size=(2,2),stride=(2,2))
#         self.norm4 = Lambda(lambda x:x)
#         self.actv4 = nn.Tanh()
#         self.pool4 = nn.AdaptiveAvgPool2d((1,2))

    def forward(self,X):
        X = self.pool1(self.actv1(self.norm1(self.conv1(X))))
        X = self.pool2(self.actv2(self.norm2(self.conv2(X))))
        X = self.pool3(self.actv3(self.norm3(self.conv3(X))))
#         X = self.pool4(self.actv4(self.norm4(self.conv4(X))))
        X = torch.flatten(X, 1)
        
        return X
    
# summary(Encoder([64,128,256,512]),(1,65,501),batch_size=bsz,device='cpu')    

In [ ]:
class Classifier(nn.Module):
    """
    Linear Classifier (Double Layer)

    Hint: user nn.Linear as Classifier (Single Layer)
    """
    def __init__(self,input_shape,hidden_shape,output_shape,norm=False):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(input_shape,hidden_shape)
        self.linear2 = nn.Linear(hidden_shape,output_shape)
        if norm == True:
            self.norm = Lambda(lambda x: F.normalize(X,dim=1))
        else:
            self.norm = Lambda(lambda x:x)

    def forward(self,X):
        X = F.leaky_relu(self.linear1(X),inplace=True)
        X = F.dropout(X,0.2)
        X = self.linear2(X)
        X = self.norm(X)
        return X

# summary(Classifier(1024,128,7),(1024,),batch_size=bsz,device='cpu')

In [ ]:
from models import ED_module

def create_model(enc=None):
    enc = Encoder([64,128,256,512])
    clf = Classifier(1024,128,7)
    model = ED_module(encoder=enc,decoder=clf)
    return model


model = create_model()

In [ ]:
from losses import SupConLoss


# criterion = SupConLoss(temperature=0.1,base_temperature=0.1) # Contrastive loss
criterion = nn.CrossEntropyLoss() # Cross entropy 

In [ ]:
optimizer = torch.optim.SGD(list(model.parameters()), lr=0.0005)

In [ ]:
from train import train

model, record = train(model,train_loader,criterion,optimizer,fine_tune_epochs,1,test_loader,parallel)

In [ ]:
from train import make_directory

def record_log(mode,epochs,record,cmtx=None,cls=None):
    if mode == 'pretrain':
        path = make_directory(exp_name+'_pretrain',epoch=epochs,filepath=PATH+'/record/')
        pd.DataFrame(record['train'],columns=['train_loss']).to_csv(path+'_loss.csv')
    elif mode == 'finetuning':
        path = make_directory(exp_name+'_finetuning',epoch=epochs,filepath=PATH+'/record/')
        pd.DataFrame(record['train'],columns=['train_loss']).to_csv(path+'_loss.csv')
        pd.DataFrame(record['validation'],columns=['validation_accuracy']).to_csv(path+'_accuracy.csv')
        cls.to_csv(path+'_report.csv')
        cmtx.to_csv(path+'_cmtx.csv')
    return

record_log('pretrain',pre_train_epochs,record)

In [ ]:
from train import evaluation

cmtx,cls = evaluation(model,test_loader,label_encoder=lb)

In [ ]:
def record_log(mode,epochs,record,cmtx=None,cls=None):
    if mode == 'pretrain':
        path = make_directory(exp_name+'_pretrain',epoch=epochs,filepath=PATH+'/record/')
        pd.DataFrame(record['train'],columns=['train_loss']).to_csv(path+'_loss.csv')
    elif mode == 'finetuning':
        path = make_directory(exp_name+'_finetuning',epoch=epochs,filepath=PATH+'/record/')
        pd.DataFrame(record['train'],columns=['train_loss']).to_csv(path+'_loss.csv')
        pd.DataFrame(record['validation'],columns=['validation_accuracy']).to_csv(path+'_accuracy.csv')
        cls.to_csv(path+'_report.csv')
        cmtx.to_csv(path+'_cmtx.csv')
    return

record_log('finetuning',fine_tune_epochs,record,cmtx,cls)

In [ ]:
save('finetuning',model,optimizer,fine_tune_epochs)